In [31]:
# the imports used in A2
import re
# import json
from glob import glob
import os
from io import StringIO
from itertools import groupby
import pickle

import numpy as np
import bs4
%matplotlib inline
import matplotlib.pyplot as plt
# Imports that might help with various functionality
import functools
import operator

# Additional imports from A3
from __future__ import print_function
import math
from collections import defaultdict
from collections import Counter
from nltk.tokenize import TreebankWordTokenizer
import Levenshtein  # package python-Levenshtein

# Additional imports from A5
from __future__ import print_function
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from numpy import linalg as LA
import json
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
import sys
# Ensure that your kernel is using Python3
assert sys.version_info.major == 3

In [18]:
# import the json
import json
with open("updated_data.json", "r") as f:
    women_summaries = json.load(f)

Loaded 1378 women
Each woman's entry is a dictionary with the following keys...
dict_keys(['summary', 'name'])


In [21]:
# creates a list of women to keep as a global variable
women_names = list()
for i in range(len(women_summaries)):
    name = women_summaries[i]['name']
    end_index = name.find('(')
    if end_index != -1 and name[: (end_index-1)] not in women_names :
        women_names.append(name[: (end_index-1)])
    elif end_index == -1 and name not in women_names :
        women_names.append(name)
num_women = len(women_names)
# print(women_names)

### Assigning an index for each woman:

In [6]:
# LATER for optimization: Assigns an index for each woman without needing a new list 

# # PEGAH
# # Here, we will assign an index for each woman. This index will help us access data in numpy matrices.
# # woman_to_index = movie_id_to_index
# woman_to_index = {woman:index for index, woman in enumerate([d['name'] for d in women_summaries])}


# # and because it might be useful...
# woman_name_to_index = {name:woman_to_index[name] for name in [d['name'] for d in women_summaries]}
# woman_index_to_name = {v:k for k,v in woman_name_to_index.items()}

# print("The index of \"{}\" is {}".format(women_summaries[7]['name'], woman_name_to_index[women_summaries[7]['name']]))

Tokenizer below

In [7]:
# # creates a dictionary {'name', [list of words in the summary]}
# women_dict_1sent = dict()
# # for i in range(len(women_summaries)):
# #     name = women_summaries[i]['name']
# for i in range(len(women_names)):
#     name = women_names[i]
#     summary = women_summaries[i]['summary']
#     # to do: 
#     # hardcoded the words that we're looking for
#     # change to regex later time
#     index_was = summary.find('was ')
#     index_is = summary.find('is ')
#     index_currently = summary.find('currently ')
#     if (index_was == -1 and index_is == -1):
#         summary2 = summary[index_currently:]
#         index_period = summary2.find('.')
#         first_sent = summary2[:index_period]
#     elif (index_was == -1 and index_currently == -1):
#         summary2 = summary[index_is:]
#         index_period = summary2.find('.')
#         first_sent = summary2[:index_period]
#     elif (index_is == -1 and index_currently == -1):
#         summary2 = summary[index_was:]
#         index_period = summary2.find('.')
#         first_sent = summary2[:index_period]
#     else :
#         index_min = min(index_is, index_was)
#         summary2 = summary[index_min:]
#         index_period = summary2.find('.')
#         first_sent = summary2[:index_period]
#     # might want to change if we use ML that finds proper nouns
#     # implement a stemmer
#     first_sent_lower = first_sent.lower()
#     tokenizer = TreebankWordTokenizer()
#     first_sent_tok = tokenizer.tokenize(first_sent_lower)
#     filtered_first_sent = [w for w in first_sent_tok if w not in stopwords.words('english')]
#     women_dict_1sent[name] = filtered_first_sent
    
# print(women_dict_1sent)

    

# Cosine sim

## TF-IDF vectorizer

In [24]:
n_feats = 5000 # QUESTION: What does this do?
doc_by_vocab = np.empty([len(women_summaries), n_feats])

def build_vectorizer(max_features, stop_words, max_df=0.5, min_df=1, norm='l2'):
    """Returns a TfidfVectorizer object
    
    Params: {max_features: Integer,
             max_df: Float,
             min_df: Float,
             norm: String,
             stop_words: String}
    Returns: TfidfVectorizer
    """
    # YOUR CODE HERE
    return TfidfVectorizer(stop_words=stop_words, max_features=max_features, max_df=max_df, min_df=min_df, norm=norm)
    
tfidf_vec = build_vectorizer(n_feats, "english")
doc_by_vocab = tfidf_vec.fit_transform([d['summary'] for d in women_summaries]).toarray()
index_to_vocab = {i:v for i, v in enumerate(tfidf_vec.get_feature_names())}

In [25]:
def get_sim(wom1, wom2, input_doc_mat, women_names):
    """Returns a float giving the cosine similarity of 
       the two movie transcripts.
    
    Params: {mov1: String,
             mov2: String,
             input_doc_mat: Numpy Array,
             movie_name_to_index: Dict}
    Returns: Float (Cosine similarity of the two movie transcripts.)
    """
    # YOUR CODE HERE
    
    a = input_doc_mat[women_names.index(wom1)] #this gives us the [......] freq of all words
    b = input_doc_mat[women_names.index(wom2)] 
    return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))

In [26]:
"""Check that get_sim returns the correct output"""
print("Similarity: Mary Anning vs. Lilias Armstrong")
print("======")
test1 = get_sim('Lilias Armstrong', 'Mary Anning', doc_by_vocab, women_names)
print(test1)
assert test1 < 1 and test1 > 0
print("")

# print("Similarity: Star Wars vs. Star Trek: Generations")
# print("======")
# test2 = get_sim('star wars', 'star trek: generations', doc_by_vocab, movie_name_to_index)
# print(test2)
# assert test2 < 0.25 and test2 > 0.20

Similarity: Mary Anning vs. Lilias Armstrong
0.04431217025859715



In [27]:
def build_woman_sims_cos(n_wom, input_doc_mat,input_get_sim_method, women_names):
    """Returns a movie_sims matrix of size (num_movies,num_movies) where for (i,j):
        [i,j] should be the cosine similarity between the movie with index i and the movie with index j
        
    Note: What you set the diagonal to doesn't really matter, 
    but you should set it to 1 to indicate that all movies are trivially perfectly similar to themselves.
    
    Params: {n_mov: Integer,
             movie_index_to_name: String,
             input_doc_mat: Numpy Array,
             movie_name_to_index: Dict,
             input_get_sim_method: Function}
    Returns: Numpy Array 
    """
    
    women_sims = np.ones((n_wom, n_wom)) #instantiate array
    for i in range(0, n_wom):
        for j in range(0, n_wom):
            women_sims[i,j] = input_get_sim_method(women_names[i], women_names[j], input_doc_mat, women_names)
    return women_sims
    raise NotImplementedError()

In [28]:
woman_sims_cos = build_woman_sims_cos(num_women,doc_by_vocab, get_sim, women_names)


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


In [32]:
# INCOMPLETE
# # From A4
# def index_search(query, sim_matrix, input_doc_mat):
    
#     tokenizer = TreebankWordTokenizer()
#     q_tokens = tokenizer.tokenize(query)
#     q = {} #vector of weights by term for query
#     results = defaultdict(int) # document_id : score
    
#     vocab = index_to_vocab.values() # can be done globally too i guess
    
#     # count freq within query
#     for tok in q_tokens:
#         if tok in vocab:
#             q[tok] = q_tokens.count(tok)
    
#     for q_i in q:
#         if q_i in vocab:
#             for rel_doc 
            
    
    
#     # For token in query
#     accum = 0
#     for tok in q_tokens:
#         if tok in vocab: # if the token is a word in our summary
            
#             accum += tfidf_vec[][]
#         else: return "NO!!!!!!!"
            

In [35]:
# print(index_search("was a biologist", woman_sims_cos))

## end cosine sim stuff pegah did

In [297]:
def sumWords(text): 
    wordDict = dict()
    for word in text :
        if word in wordDict :
            wordDict[word] = wordDict[word] + 1
        else :
            wordDict[word] = 1
    return (wordDict)

In [303]:
# create the global variable of what is equivalent to good_types
unique_word_lst = list()
for woman in women_dict_1sent :
    summary = women_dict_1sent[woman]
    for word in summary :
        if word not in unique_word_lst :
            unique_word_lst.append(word)
        

In [311]:
# creates a np array where the rows are women according to the list of women names
# columns are rows according to the list of unique words
def create_word_freq_array(input_women_dict_1sent, input_women_names, input_num_women, input_unique_word_lst):
    dict_freq = dict()

    for woman in input_women_dict_1sent :
        dict_freq[woman] = sumWords(input_women_dict_1sent[woman])
    
    np_with_freq = np.zeros(shape = (len(dict_freq), len(input_unique_word_lst)))
    i = 0
    for woman in input_women_names :
#         print(woman)
        if woman in dict_freq :
            j = 0
            for word in input_unique_word_lst :
                if word in dict_freq[woman] :
#                     print(j)
                    np_with_freq[i][j] = dict_freq[woman][word]
                j = j + 1
            i = i + 1
#     print(np_with_freq[0][0])
    return np_with_freq
#     print(input_unique_word_lst.index('chogha'))
    
        

In [325]:
word_freq_array = create_word_freq_array(women_dict_1sent, women_names, len(women_dict_1sent), unique_word_lst)

In [326]:
# The file is ordered from the homeworks that we've worked on
# so that we can easily go back to the specific assignment if we need to

In [329]:
def create_j_sim_mat(input_num_women, input_word_freq_array, input_unique_word_lst):
    arr = np.zeros(shape = (input_num_women, input_num_women))
    
    for (i, woman1) in enumerate(input_word_freq_array) :
        for (j, woman2) in enumerate(input_word_freq_array) :
            s1 = np.nonzero(woman1)
            s2 = np.nonzero(woman2)
            intersect = np.intersect1d(s1, s2)
            union = np.union1d(np.array(s1).flatten(), np.array(s2).flatten())
            if len(union) > 0 :
                arr[i][j] = len(intersect)/len(union)
                
    np.fill_diagonal(arr, 0)
    
#     print(np.sum(arr[100:]))
    return arr
    
    

In [330]:
j_sim_mat = create_j_sim_mat(len(women_names), word_freq_array, unique_word_lst)

In [ ]:
# make dictionary of top 5 women for each woman
def create_top_5_dict(j_sim_mat):
    
    

In [ ]:
# Homework 3 looked at Levenshtein distance (probably not a good idea imo)

In [ ]:
# Homework 4 looked at cosine similarity, which we can consider on top of jaccard